In [1]:
import os

import anthropic
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


SCOPES = ['https://www.googleapis.com/auth/calendar']



if os.path.exists('./res/token.json'):
    creds = Credentials.from_authorized_user_file('./res/token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './res/credentials.json', SCOPES
    )
    creds = flow.run_local_server(port=0)
    with open('./res/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)

client = anthropic.Anthropic(
    api_key=os.environ['ANTHROPIC_API_KEY']
)

### Multi-Turn Chat
- User-Assistant 간의 대화 히스토리를 저장 후 LLM API에 입력
  - 이전 대화 내용을 기억하고 기존 대화 맥락을 참고해서 알맞는 답변 생성
- 턴 수가 많아질수록 LLM API 답변 퀄리티가 낮아지거나 일부 내용이 있더라도 찾아내지 못 할 가능성 존재
- 입력 토큰 수 증가

In [2]:
messages = []

In [3]:
PROMPT = '2024년 8월 14일 12시부터 1시간 짜리 점심 일정 생성해주세요.'

# 1. 함수 생성
def create_event(summary, start, end):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start,
            'timeZone': 'Asia/Seoul',
        },
        'end': {
            'dateTime': end,
            'timeZone': 'Asia/Seoul',
        }
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print('Event created: %s' % (event.get('htmlLink')))
    return event

# 2. tools 정의
tools = [
    {
        "name": "create_event",
        "description": "Create new Google Calender Event",
        "input_schema": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "Name of Google Calender Event"
                },
                "start": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                },
                "end": {
                    "type": "string",
                    "description": "Starting date of Google Calender Event in UTC+9 Time i.e. 2024-08-08T09:00:00+09:00"
                }
            },
            "required": ["summary"]
        }
    }
]

# 3. LLM API 호출
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": PROMPT
        }
    ],
)
messages.append(
    {
        "role": "user",
        "content": PROMPT
    }
)

# 4. (필요 시) Function Call 실행
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
event = create_event(summary, start, end)

# 5. LLM API 최종 호출
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": PROMPT
        },
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": PROMPT
                },
                {
                    "type": "tool_use",
                    "id": response.content[1].id,
                    "name": "create_event",
                    "input": {"summary": summary, "start": start, "end": end}
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": response.content[1].id,
                    "content": f"Event created: {event.get('htmlLink')}"
                }
            ]
        }
    ],
)
messages.append(
    {
        "role": "assistant",
        "content": response.content[0].text
    }
)

Event created: https://www.google.com/calendar/event?eid=dnZjZm91OTY1cjkwbGwxN2ZibXFuM280ZDAgcmVpbjIwLmFpQG0


In [4]:
messages

[{'role': 'user', 'content': '2024년 8월 14일 12시부터 1시간 짜리 점심 일정 생성해주세요.'},
 {'role': 'assistant',
  'content': '네, 2024년 8월 14일 12시부터 1시간 동안의 점심 일정이 성공적으로 생성되었습니다. 구글 캘린더에 "점심 일정"이라는 이름으로 이벤트가 추가되었습니다. \n\n일정 세부 정보:\n- 날짜: 2024년 8월 14일 (수요일)\n- 시간: 오후 12:00 ~ 오후 1:00 (1시간)\n- 제목: 점심 일정\n\n이 일정은 구글 캘린더에서 확인하실 수 있습니다. 추가로 수정이나 다른 일정 관련 요청이 있으시면 말씀해 주세요.'}]

In [4]:
print(response.content[0].text)

점심 일정이 성공적으로 생성되었습니다. 일정 세부 사항은 다음과 같습니다:

- 제목: 점심
- 날짜: 2024년 8월 1일
- 시작 시간: 오후 12:00
- 종료 시간: 오후 1:00
- 기간: 1시간

Google 캘린더에 해당 일정이 추가되었으며, 링크를 통해 확인하실 수 있습니다. 추가로 수정하거나 확인하고 싶은 사항이 있으시면 말씀해 주세요.


In [5]:
messages

[{'role': 'user', 'content': '2024년 8월 1일 12시부터 1시간 짜리 점심 일정 생성해주세요.'},
 {'role': 'assistant',
  'content': '점심 일정이 성공적으로 생성되었습니다. 일정 세부 사항은 다음과 같습니다:\n\n- 제목: 점심\n- 날짜: 2024년 8월 1일\n- 시작 시간: 오후 12:00\n- 종료 시간: 오후 1:00\n- 기간: 1시간\n\nGoogle 캘린더에 해당 일정이 추가되었으며, 링크를 통해 확인하실 수 있습니다. 추가로 수정하거나 확인하고 싶은 사항이 있으시면 말씀해 주세요.'}]

In [5]:
new_message = "제가 이전에 어떤 요청을 드렸나요?"

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=[
        {
            "role": "user",
            "content": new_message
        }
    ],
)
print(response.content[0].text)

죄송합니다만, 이전 대화 내용을 확인할 수 없어 귀하께서 어떤 요청을 하셨는지 정확히 알 수 없습니다. 이 대화는 새로 시작된 것으로 보입니다.

혹시 특정 요청이나 주제에 대해 기억나시는 것이 있다면 말씀해 주시겠습니까? 그렇게 하시면 제가 더 잘 도와드릴 수 있을 것 같습니다. 아니면 새로운 요청이나 질문이 있으시다면 언제든 말씀해 주세요.


In [6]:
new_message = "저의 가장 최근 요청은 어떤 내용이었나요?"

messages.append(
    {
        "role": "user",
        "content": new_message
    }
)

response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=messages,
)
print(response.content[0].text)

귀하의 가장 최근 요청은 다음과 같습니다:

"2024년 8월 14일 12시부터 1시간 짜리 점심 일정 생성해주세요."

이 요청에 따라 저는 구글 캘린더에 해당 일정을 생성하였습니다. 구체적으로:

1. 날짜: 2024년 8월 14일
2. 시간: 오후 12:00부터 오후 1:00까지 (1시간 동안)
3. 제목: "점심 일정"

이 일정이 성공적으로 생성되었음을 알려드렸습니다. 혹시 이 일정에 대해 추가로 수정하거나 다른 요청사항이 있으신가요?
